In [ ]:
import numpy as np

# Function and testing

## **Map**

### getPositionAround(x,y)

In [ ]:
def getPositionAround(x,y):
  minX = 0
  minY = 0
  maxX = 10
  maxY = 5
  var = [-1,0,1]
  around = []

  for i in range(3):
    for j in range(3):
      tempX = x+var[i]
      tempY = y+var[j]
      if (tempX==x and tempY==y) or (tempX<minX or tempY<minY) or (tempX>maxX or tempY>maxY):
        continue
      around.append((tempX, tempY))
  return around

### setMap(lowPat, midPat, hardPat, wall)

In [ ]:
def setMap(poslow, posMid, posHard, wall):
  map = np.zeros((11,6),np.int8)
  mask = np.zeros((11,6),np.int8)
  position = [poslow, posMid, posHard, wall]
  behavior = [1,2,3,0]
  # เซ็ทแหน่งของผู้ป่วย
  for i, pos in enumerate(position):
    for j in range(len(pos)):
      # ตำแหน่งปัจจุบัน
      x = pos[j][0]
      y = pos[j][1]
      # หารอบๆ
      ard = getPositionAround(x,y)
      # เซ็ทค่าตำแหน่งปัจจุบัน
      if i==0:
        map[x][y] = 97
        mask[x][y] = 97
      elif i==1:
        map[x][y] = 98
        mask[x][y] = 98
      elif i==2:
        map[x][y] = 99
        mask[x][y] = 99
      elif i==3:
        map[x][y] = 111
        mask[x][y] = 111

      around = getPositionAround(x,y)
      for ard in around:
        ardX = ard[0]
        ardY = ard[1]
        if map[ardX,ardY]== 97 or map[ardX,ardY]== 98 or map[ardX,ardY]== 99:
          continue
        else:
          map[ardX,ardY]+=behavior[i]
  return map, mask

### deleteMapWeight(map,pos)

In [ ]:
def deleteMapWeight(map,pos):
  x = pos[0]
  y = pos[1]
  behavior = [1,2,3]
  if map[x][y]==97 or map[x][y]==98 or map[x][y]==99:
    around = getPositionAround(x,y)
    for pos in around:
      ardX = pos[0]
      ardY = pos[1]
      if map[ardX][ardY]==97 or map[ardX][ardY]==98  or map[ardX][ardY]==99:
        continue
      else:
        if map[x][y]==97 and map[ardX][ardY]!=-1:
          map[ardX][ardY]-=behavior[0]
        elif map[x][y]==98 and map[ardX][ardY]!=-1:
          map[ardX][ardY]-=behavior[1]
        elif map[x][y]==99 and map[ardX][ardY]!=-1:
          map[ardX][ardY]-=behavior[2]

    map[x][y]=-1
  else:
    print('not position of patient')

  return map

### heaviestCase(map)

In [ ]:
def heaviestCase(map):
  x99,y99 = np.where(map==99)
  x98,y98 = np.where(map==98)
  x97,y97 = np.where(map==97)
  heaviest = 0

  if x99.size!=0:
    # print('heaviest case is 99')
    heaviest = 99
  elif x98.size!=0:
    # print('heaviest case is 98')
    heaviest = 98
  elif x97.size!=0:
    # print('heaviest case is 97')
    heaviest = 97
  return heaviest

## **Destination**

### astar(maze, start, end)

In [ ]:
class Node():
    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position
        
        self.g = 0
        self.h = 0
        self.f = 0
    def __eq__(self, other):
        return self.position == other.position

def astar(maze, start, end):
    # Create start and end node
    start_node = Node(None, start)
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node(None, end)
    end_node.g = end_node.h = end_node.f = 0

    # Initialize both open and closed list
    open_list = []
    closed_list = []
    # Add the start node
    open_list.append(start_node)

    # Loop until you find the end
    while len(open_list) > 0:

        # Get the current node
        current_node = open_list[0]
        current_index = 0
        for index, item in enumerate(open_list):
            if item.f < current_node.f:
                current_node = item
                current_index = index

        # Pop current off open list, add to closed list
        open_list.pop(current_index)
        closed_list.append(current_node)

        # Found the goal
        if current_node == end_node:
            path = []
            current = current_node
            while current is not None:
                path.append(current.position)
                current = current.parent
            return path[::-1] # Return reversed path

        # Generate children
        children = []
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]: # Adjacent squares

            # Get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            # Make sure within range
            if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (len(maze[len(maze)-1]) -1) or node_position[1] < 0:
                continue

            # Make sure walkable terrain
            if maze[node_position[0]][node_position[1]] != 0:
                continue

            # Create new node
            new_node = Node(current_node, node_position)

            # Append
            children.append(new_node)

        # Loop through children
        for child in children:

            # Child is on the closed list
            for closed_child in closed_list:
                if child == closed_child:
                    continue

            # Create the f, g, and h values
            child.g = current_node.g + 1
            child.h = ((child.position[0] - end_node.position[0]) ** 2) + ((child.position[1] - end_node.position[1]) ** 2)
            child.f = child.g + child.h

            # Child is already in the open list
            for open_node in open_list:
                if child == open_node and child.g > open_node.g:
                    continue

            # Add the child to the open list
            open_list.append(child)

### getSetDestination(map)

In [ ]:
def getSetDestination(map):
  if heaviestCase(map)!=0:
    des = np.argwhere(map==heaviestCase(map))
    desArd = []
    max = 0
    allAround = []
    # print('destination: ',des)
    # หา index รอบๆผู้ป่วยที่ป่วยที่สุด
    for idx, pos in enumerate(des):
      x = pos[0]
      y = pos[1]
      around = getPositionAround(x,y)
      allAround.append(around)
      # print('ard: ',around)
      # หาว่าช่องไหนมันมีค่ามาที่สุด
      for ard in around:
        ardX = ard[0]
        ardY = ard[1]
        # print(ard,' ',map[ardX,ardY] )
        # print('...',ardX,',',ardY,'val: ',map[ardY,ardX])
        if map[ardX,ardY]==97 or map[ardX,ardY]==98 or map[ardX,ardY]==99:
          continue
        elif map[ardX,ardY]>max:
          max = map[ardX,ardY]
    # print('allAround: ',allAround)
    # print('max: ',max)  

    ardd = []
    for i in allAround:
      for j in i:
        ardd.append(j)
    aroundSet = list(set(ardd)) 

    for ard in aroundSet:
      ardX = ard[0]
      ardY = ard[1]
      if map[ardX,ardY]==max:
        desArd.append(ard)
    # print('candidate: ',desArd)
    return desArd
  else:
    return 0

### getbestDestination(mask, start, desSet)

In [ ]:
def getbestDestination(mask, start, desSet):
  if desSet!=0:
    bestPathLen = 999
    bestPathIndex = 0
    bestPath = []
    for i, pos in enumerate(desSet):
      x = pos[0]
      y = pos[1]
      path = astar(mask, start, (x,y))
      # print('path: ', path)
      if len(path) <= bestPathLen:
        # print('index: ', i,' len: ',len(path))
        bestPathIndex = i
        bestPathLen = len(path)
        bestPath = path
    # print('des pos: ', pos,'path: ',len(path))
    # print(bestPathIndex)
    # print(desSet[bestPathIndex])

    return desSet[bestPathIndex], bestPath
  else:
    path = astar(mask, start, (10,3))
    return (10,3), path

### getDestination(map, mask, start)
wrapper 

In [ ]:
def getDestination(map, mask, start):
  desSet = getSetDestination(map)
  des, path = getbestDestination(mask, start, desSet)
  # tuple -> list
  for i in range(len(path)):
    path[i] = list(path[i])
  return des, path

### findPatientAround(pos)

In [ ]:
def findPatientAround(pos):
  x = pos[0]
  y = pos[1]
  around = getPositionAround(x,y)
  hard = []
  mid = []
  low = []
  all = []

  for ard in around:
    x = ard[0]
    y = ard[1]
    # print(map[x][y])
    if map[x][y]==99:
      hard.append([x,y])
    elif map[x][y]==98:
      mid.append([x,y])
    elif map[x][y]==97:
      low.append([x,y])
  if len(hard)!=0:
    all += hard
  elif len(mid)!=0:
    all += mid
  elif len(low)!=0:
    all += low

  return all

### makeFinalPath(map,mask)

In [ ]:
def makeFinalPath(map,mask):
  finalPath = []
  i=0
  des = (0, 0)
  while des!=(10, 3):
    if i==0:
      start = (10,3)
    else:
      start = des
    des, path = getDestination(map, mask, start)
    targets = findPatientAround(des)
    for pos in targets:
      deleteMapWeight(map, pos)

    # print(des)
    # print(map)
    # print(path)
    # print(targets)

    for idx, target in enumerate(targets):
      pathPair = path.copy()
      if idx==0:
        pathPair.append(target)
        finalPath.append(pathPair)
      else: 
        cur = pathPair[-1]
        temp = [cur, target]
        finalPath.append(temp)
  
    if des == (10, 3):
      finalPath.append(path) 
    i+=1

  return finalPath

### patTranform(list)

In [ ]:
def patTranform(finalPath):
  temp = finalPath.copy()
  for i, row in enumerate(finalPath):
    for j, value in enumerate(row):
      # print(finalPath[i][j])
     temp[i][j] = [abs(value[0]-10),value[1]]
  return temp

# **Map 1**

In [ ]:
# map 1
lowPat = [(0,0),(5,4)]
midPat = [(1,0), (3,2)]
hardPat = [(1,5)]
wall = [(7,0), (8,0), (9,0), (10,0), (8,1), (9,1), (10,1), (8,4), (9,4), (10,4) ,(7,5) ,(8,5) ,(9,5) ,(10,5)]

In [ ]:
map, mask = setMap(lowPat, midPat, hardPat, wall)
finalPath = makeFinalPath(map,mask)
finalPath

[[[10, 3],
  [9, 3],
  [8, 3],
  [7, 4],
  [6, 4],
  [5, 3],
  [4, 4],
  [3, 4],
  [2, 4],
  [1, 5]],
 [[2, 4], [2, 3], [2, 2], [2, 1], [1, 0]],
 [[2, 1], [3, 2]],
 [[2, 1], [1, 1], [0, 0]],
 [[1, 1], [2, 2], [3, 3], [4, 3], [5, 4]],
 [[4, 3], [5, 3], [6, 3], [7, 3], [8, 3], [9, 3], [10, 3]]]

In [ ]:
patTranform(finalPath)

[[[0, 3],
  [1, 3],
  [2, 3],
  [3, 4],
  [4, 4],
  [5, 3],
  [6, 4],
  [7, 4],
  [8, 4],
  [9, 5]],
 [[8, 4], [8, 3], [8, 2], [8, 1], [9, 0]],
 [[8, 1], [7, 2]],
 [[8, 1], [9, 1], [10, 0]],
 [[9, 1], [8, 2], [7, 3], [6, 3], [5, 4]],
 [[6, 3], [5, 3], [4, 3], [3, 3], [2, 3], [1, 3], [0, 3]]]

In [ ]:
mask

array([[ 97,   0,   0,   0,   0,   0],
       [ 98,   0,   0,   0,   0,  99],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,  98,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,  97,   0],
       [  0,   0,   0,   0,   0,   0],
       [111,   0,   0,   0,   0, 111],
       [111, 111,   0,   0, 111, 111],
       [111, 111,   0,   0, 111, 111],
       [111, 111,   0,   0, 111, 111]], dtype=int8)